<a href="https://colab.research.google.com/github/TomasWW/llm-rag-demo/blob/main/rag_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages
!pip install -U -q langchain langchain-openai langchain-community qdrant-client pypdf openai

In [ ]:
import os
from google.colab import userdata
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage

# Load your OpenAI API key from Colab secrets
OPENAI_API_KEY = userdata.get('apikey')
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
OPENROUTER_API_BASE = "https://openrouter.ai/api/v1"

# Initialize LLM
llm = ChatOpenAI(model="mistralai/mistral-7b-instruct",
                 temperature=0.7,
                 openai_api_base=OPENROUTER_API_BASE
                 )

# Example query using only LLM (without RAG)
response = llm.invoke([
    HumanMessage(content="Who won the Mundial de Clubes FIFA 2025")
])
print("LLM-only response:\n", response.content)



In [ ]:
#Qdrant setup

from langchain.vectorstores import Qdrant
from qdrant_client import QdrantClient

# Connect to Qdrant vector database
DB_URL = userdata.get('dburl')
RAG_ENGINE_API_KEY = userdata.get('apikeyrag-engine')

qdrant_client = QdrantClient(
    url=DB_URL,
    api_key=RAG_ENGINE_API_KEY
)

COLLECTION_NAME = "mundial_clubes_2025"

# Delete the collection if it already exists
try:
    qdrant_client.delete_collection(collection_name=COLLECTION_NAME)
    print(f"Collection '{COLLECTION_NAME}' deleted.")
except Exception as e:
    print(f"Collection did not exist: {e}")


In [ ]:
# Load and split PDF files
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load PDF files
pdf_files = [
    "Anexo_Final de la Copa Mundial de Clubes de la FIFA 2025.pdf",
    "Copa_Mundial_de_Clubes_de_la_FIFA_2025.pdf"
]

docs = []
for file in pdf_files:
    loader = PyPDFLoader(file)
    docs += loader.load()

print(f"Loaded {len(docs)} pages from the PDF files.")

# Split text into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = splitter.split_documents(docs)
print(f"Created {len(chunks)} text chunks.")


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

# Create embeddings using a HuggingFace model
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Insert chunks into Qdrant
qdrant = Qdrant.from_documents(
    chunks,
    embeddings,
    location=DB_URL,
    api_key=RAG_ENGINE_API_KEY,
    collection_name=COLLECTION_NAME
)

print(f"Upload complete. {len(chunks)} embeddings inserted into Qdrant.")


In [ ]:
# Function to retrieve the most relevant chunks for a user question
def get_context(user_question, k=5):
    """
    Retrieve the k most relevant text chunks from Qdrant
    and concatenate them as context for the LLM.
    """
    docs = qdrant.similarity_search(user_question, k=k)
    context = "\n".join([doc.page_content for doc in docs])
    return context


In [ ]:
# User question
user_input = "Quien fue el campeon del Mundial de Clubes FIFA 2025"

# Retrieve context from vector DB
context = get_context(user_input)
print("Retrieved context:\n", context, "\n\n\n")

# Ask LLM using retrieved context
response = llm.invoke([
    HumanMessage(content=f"Using the following context:\n{context}\n\nAnswer the question: {user_input}")
])
print("RAG-enhanced response:\n", response.content)